# Lesson 2: Pandas Bootcamp Part 1 - updated for class on 1/17/24

[Acknowledgments Page](https://ds100.org/fa23/acks/)

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px

### Loading Elections Data Into a DataFrame:

Panda's [read_csv function](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) is one of the most versatile and useful functions for managing data.  

**Practice:  Load the elections data**

In [5]:
elections = pd.read_csv("data/elections.csv")


### `DataFrame` attributes: `index`, `columns`

In [ ]:
elections.index

In [ ]:
elections.columns

The `Index` column can be set to the default list of integers by calling `reset_index()` on a `DataFrame`.

# Extraction:

One of the most basic tasks for manipulating a DataFrame is to extract rows and columns of interest.   


### Label-Based Extraction Using`loc`

`loc` selects items by row and column *label*.  

`df.loc[row_labels, column_labels]`

We describe "labels" as the bolded text at the top and left of a DataFrame.




Arguments to `.loc` can be:
1. A row label and column label
2. A list.
3. A slice (syntax is inclusive of the right-hand side of the slice).

In [ ]:
# Here's how we can select all rows and just the Year and Party columns from the elections dataframe.
# Note we use the ellipsis (:) in the first entry because we want to select all rows

elections.loc[:,["Year","Party"]]

In [ ]:
# Selection by a list

elections.loc[[87, 25, 179], ["Year", "Candidate", "Result"]]

In [ ]:
# Selection by a list and a slice of columns
elections.loc[[87, 25, 179], "Popular vote":"%"]

In [ ]:
# Extracting all rows using a colon
elections.loc[:, ["Year", "Candidate", "Result"]]

In [ ]:
# Extracting all columns using a colon
elections.loc[[87, 25, 179], :]

In [ ]:
# Selection by a list and a single-column label
elections.loc[[87, 25, 179], "Popular vote"]

In [ ]:
# Note that if we pass "Popular vote" in a list, the output will be a DataFrame
elections.loc[[87, 25, 179], ["Popular vote"]]

In [ ]:
# Selection by a row label and a column label
elections.loc[0, "Candidate"]

#### Integer-Based Extraction Using `iloc`

`iloc` selects items by row and column *integer* position.

Arguments to `.iloc` can be:
1. A list.
2. A slice (syntax is exclusive of the right hand side of the slice).
3. A single value.


In [ ]:
# Select the rows at positions 1, 2, and 3.
# Select the columns at positions 0, 1, and 2.
# Remember that Python indexing begins at position 0!
elections.iloc[[1, 2, 3], [0, 1, 2]]

In [ ]:
# Index-based extraction using a list of rows and a slice of column indices
elections.iloc[[1, 2, 3], 0:3]

In [ ]:
# Selecting all rows using a colon
elections.iloc[:, 0:3]

In [ ]:
elections.iloc[[1, 2, 3], 1]

In [ ]:
# Extracting the value at row 0 and the second column
elections.iloc[0,1]

#### Context-dependent Extraction using `[]`

We could technically do anything we want using `loc` or `iloc`. However, in practice, the `[]` operator is often used instead to yield more concise code.

`[]` is a bit trickier to understand than `loc` or `iloc`, but it achieves essentially the same functionality. The difference is that `[]` is *context-dependent*.

`[]` only takes one argument, which may be:
1. A slice of row integers.
2. A list of column labels.
3. A single column label.


If we provide a slice of row numbers, [start:stop], we get all rows with those integer positions.  While the element at the start index is included, the stop index is not included, so that the number of elements in the result is stop - start. 

In [ ]:
elections[3:7]

If we provide a list of column names, we get the listed columns.

In [ ]:
elections[["Year", "Candidate", "Result"]]

And if we provide a single column name we get back just that column, stored as a `Series`.

In [ ]:
elections["Candidate"]

### Multi-indexed DataFrames

You can also define multiple indexes for the same DataFrame.  This is useful when you need more than one column to specify the granularity of the data.  
For example, if we wanted to use both `Year` and `Party` as our indices we would do this as follows:

In [ ]:
elections_multindex = elections.set_index(["Year","Party"])

In [ ]:
elections_multindex.head()

### Accessing Data in Multi-indexed DataFrames:

Now, to access data we can use `.loc` where the first entry is a tuple: (year, party):


In [ ]:
elections_multindex.loc[(1828,"Democratic"),:]

Notice, we got a warning above.  This just means that your index is not sorted. pandas depends on the index being sorted (in this case, lexicographically, since we are dealing with string values) for optimal search and retrieval. A quick fix would be to sort your DataFrame in advance using DataFrame.sort_index. This is especially desirable from a performance standpoint if you plan on doing multiple such queries in tandem:

In [ ]:
elections_multindex = elections_multindex.sort_index()
elections_multindex.loc[(1828,"Democratic"),:]

## Setting a New Index:

Suppose we want to know how many elections Andrew Jackson ran in.

**Practice:** Set the elections index to be Candidate.

In [ ]:
elections = elections.set_index("Candidate")
elections

**Practice:Select only the rows when Andrew Jackson ran in an election**

In [ ]:
elections.loc["Andrew Jackson"]

**Practice:  Reset the index (to the default integer indices)**

In [ ]:
elections = elections.reset_index()

**Practice:  Create a new dataframe that is just the first 10 rows of the elections dataframe**

In [ ]:
elections_first_10 = elections.head(10)
elections_first_10

## Boolean Arrays

In [ ]:
a = np.array([True, False, True, False, True, False, False, False, False, False])

In [ ]:
# What happens when you sum a boolean array?
a.sum()

In [ ]:
# What happens if you put a boolean array as an input to the .loc or [] operator?

In [ ]:
elections_first_10[a]

In [ ]:
elections_first_10[[True, False, True, True, False, False, True, False, True, False]]

## Conditional Selection

By passing in a sequence (list, array, or `Series`) of boolean values, we can extract a subset of the rows in a `DataFrame`. We will keep *only* the rows that correspond to a boolean value of `True`.


**Practice:  Use Conditional Selection to Extract all rows from the elections DataFrame where the percentage of popular votes was greater than 50%**

In [ ]:
# First, use a logical condition to generate a boolean Series:
# (another name for this logical operator is a "Boolean Mask")
logical_operator = elections["%"]>50

logical_operator

In [ ]:
# Then, use this boolean array to filter the DataFrame
elections[logical_operator]


### Bitwise Operators

To filter on multiple conditions, we combine boolean operators using **bitwise comparisons**.

Symbol | Usage      | Meaning 
------ | ---------- | -------------------------------------
~    | ~p       | Returns negation of p
&#124; | p &#124; q | p OR q
&    | p & q    | p AND q
^  | p ^ q | p XOR q (exclusive or)

**Practice: Extract all rows from the elections DataFrame when Andrew Jackson was elected president**

In [7]:
# OPTION 1: 
elections[(elections["Candidate"]=="Andrew Jackson") & (elections["Result"]=="win")]

,Year,Candidate,Party,Popular vote,Result,%
182,1832,Andrew Jackson,Democratic,702735,win,54.574789
185,1828,Andrew Jackson,Democratic,642806,win,56.203927


In [9]:
# OPTION 2: 
elections.loc[(elections["Candidate"]=="Andrew Jackson") & (elections["Result"]=="win")]

,Year,Candidate,Party,Popular vote,Result,%
182,1832,Andrew Jackson,Democratic,702735,win,54.574789
185,1828,Andrew Jackson,Democratic,642806,win,56.203927


In [11]:
# OPTION 3: Reset index and then use loc
elections= elections.set_index(["Candidate", "Result"])
elections.loc[("Andrew Jackson", "win"),:]

/var/folders/qz/jk1bsv4975j0px690w5t5wqw0000gp/T/ipykernel_8628/227001221.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  elections.loc[("Andrew Jackson", "win"),:]


Year       Party  Popular vote          %
Candidate      Result                                           
Andrew Jackson win     1832  Democratic        702735  54.574789
               win     1828  Democratic        642806  56.203927

In [13]:
# reset index for further examples below

elections = elections.reset_index()

### Another Selection Option:  Query

Read the documentation for query:
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html

Use query to select all rows where the Candiate was John Quincy Adams OR the Popular Vote was greater than 70,000,000


In [ ]:
elections.query("Candidate=='John Quincy Adams' | `Popular vote` > 70000000")

**Practice: Use Query to Extract all rows from the elections DataFrame when Andrew Jackson was elected president**

In [ ]:
...

**Practice: Use Query to Extract all rows from the elections DataFrame where the percentage of popular votes was greater than 50 AND the candidate lost**

In [ ]:
...

## Adding, Removing, and Modifying Columns

### Adding or Modifying a Column
To add (or modify an existing) column, use `.assign()`

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html 


Syntax:

`df = df.assign(new_col_name = new_col_values)`


In [ ]:
# Add a column called frac_voters with the fraction of voters who voted in each election
elections = elections.assign(frac_voters = elections["%"]/100)

elections

**Practice:  Add a new column to elections called "TotVoters" that gives the total number of people who voted in that particular election**

In [ ]:
...

elections

### Rename a Column Name
Rename a column using the `.rename()` method.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html


Rename "TotVoters to "Total_Voters":

In [ ]:
elections = elections.rename(columns = {"TotVoters": "Total_Voters"})

elections

### Delete a Column
Remove a column using `.drop()`
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html 


Drop the columns "frac_voters" and "Total_Voters":

In [ ]:
elections = elections.drop(columns=["frac_voters","Total_Voters"])
elections

## Useful Utility Functions

### `NumPy`

`NumPy` functions are compatible with Series objects in `pandas`. 

In [ ]:
import numpy as np

np.mean(elections["Popular vote"])

In [ ]:
# Max 

np.max(elections["Popular vote"])

### Built-In `pandas` Methods

There are many, *many* utility functions built into `pandas`, far more than we can possibly cover in lecture. You are encouraged to explore all the functionality outlined in the `pandas` [documentation](https://pandas.pydata.org/docs/reference/index.html).

#### Useful Python Functions

`len(series)`

`len(df)`



#### Useful Series Utility Functions

`series.unique()`

`series.sort_values()`

`series.value_counts()`

`series.isna()`



#### Useful DataFrame Utility Functions

`df.shape`

`df.info()`

`df.describe()`

`df.sort_values()`

`df.value_counts()`

`df.isna()`





#### Useful Python Function:  len()

In [ ]:
len(elections["Party"])

In [ ]:
len(elections)

#### Useful Utility Functions for Series (i.e. Individual Columns of DataFrame)

Run the cells below to explore what these utility functions do:

In [ ]:
elections["Party"].unique()

In [ ]:
len(elections["Party"].unique())

In [ ]:
elections["Candidate"].sort_values()

In [ ]:
elections["Candidate"].sort_values(ascending=False)

In [ ]:
elections["Candidate"].value_counts()

In [ ]:
elections["Candidate"].isna()

In [ ]:
elections[elections["Candidate"].isna()]

In [ ]:
sum(elections["Candidate"].isna())

#### Useful Utility Functions for DataFrames
Run the cells below to explore what these utility functions do:

In [ ]:
elections.shape

In [ ]:
elections.info()

In [ ]:
elections.describe()

In [ ]:
elections.sort_values(by="Year")

In [ ]:
elections.value_counts()

In [ ]:
elections[["Candidate","Party"]].value_counts()

In [ ]:
elections.isna()

In [ ]:
elections.isna().sum()